In [1]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [2]:
%gui qt5
%matplotlib qt5

from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
import matplotlib.colors as mpl_colors
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si
from apsuite.dynap import DynapXY
from apsuite.optics_analysis import TuneCorr

In [3]:
model = si.create_accelerator()
model.cavity_on = True
model.radiation_on = False
dynap = DynapXY(model)

In [18]:
tunecorr = TuneCorr(model, 'SI')

In [19]:
tunecorr.correct_parameters([49.186188, 14.2300])

1

In [20]:
twi, _ = pyaccel.optics.calc_twiss(dynap._acc)
twi.mux[-1]/2/np.pi, twi.muy[-1]/2/np.pi

(49.18574477838651, 14.230530180928666)

In [4]:
dynap.params.turn_by_turn = True
dynap.params.x_nrpts = 70
dynap.params.y_nrpts = 30
dynap.params.x_min = -0.012
dynap.params.x_max = 0.00
dynap.params.y_min = 0
dynap.params.y_max = 0.004
print(dynap)

nrturns      : 512
turn_by_turn : True
x_nrpts      : 70
y_nrpts      : 30
x_min [m]    : -0.012
x_max [m]    : 0
y_min [m]    : 0
y_max [m]    : 0.004
de_offset    : 0
xl_off [rad] : 1e-05
yl_off [rad] : 1e-05
intnux       : 49.00 (for graphs)
intnuy       : 14.00 (for graphs)



In [17]:
dynap.do_tracking()

In [18]:
dynap.process_data()

In [24]:
# dynap.save_data('/home/fernando/delta_tune2.pickle')
dynap.load_and_apply('/home/fernando/delta_tune2.pickle')
dynap.process_data()

In [5]:
dynap.load_and_apply('/home/fernando/nominal.pickle')
dynap.process_data()

# Diffusion Plot

In [14]:
bounds = [
    dynap.params.intnux, dynap.params.intnux + 0.5,
    dynap.params.intnuy, dynap.params.intnuy + 0.5]
orders = 6
symmetry = 1
resons = dynap.calc_resonances_for_bounds(bounds, orders=orders, symmetry=symmetry)

In [35]:
map2xy = dynap.map_resons2real_plane(resons, maxdist=2e-3, min_diffusion=1e-5)
numb = [x.size for x in map2xy]
filter_resons = []
for i, (res, nu) in enumerate(zip(resons, numb)):
    if nu > 15:
        print(f'{i:2}: {res}  {nu:3}')
        filter_resons.append(res)

to_remove = [-2]
for idx in sorted(to_remove, reverse=True):
    del filter_resons[idx]

print()
for i, reson in enumerate(filter_resons):
    print(f'{reson}')

15: (4, 2, 225)   20
19: (2, 4, 155)   17
38: (0, 4, 57)   22
41: (0, 6, 85)   47
52: (-4, 1, -182)   60
63: (3, 3, 190)   17
70: (-2, 1, -84)   35
90: (1, 2, 78)   27

(4, 2, 225)
(2, 4, 155)
(0, 4, 57)
(0, 6, 85)
(-4, 1, -182)
(3, 3, 190)
(1, 2, 78)


In [36]:
f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=5e-3, min_diffusion=1e-5)
# f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=1e-3)
f.show()
# f = dynap.make_figure_diffusion(contour=False)

# X and Y map to tune plot

In [29]:
f = dynap.make_figure_xandy_map_in_tune_plot(resons)